### Local Analyses
They work on two or more arrays that are the same size, and an algebraic equation
is applied to each set of pixel locations

#### Compute NDVI

In [ ]:
import os
import numpy as np
from osgeo import gdal

In [ ]:
data_dir = r'C:\Personale\atm\reps\gis-programming\osgeopy-data'

In [ ]:
def make_raster(in_ds, fn, data, data_type, nodata=None):
    """Create a one-band GeoTIFF.

    in_ds     - datasource to copy projection and geotransform from
    fn        - path to the file to create
    data      - NumPy array containing data to write
    data_type - output data type
    nodata    - optional NoData value
    """
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(
        fn, in_ds.RasterXSize, in_ds.RasterYSize, 1, data_type)
    out_ds.SetProjection(in_ds.GetProjection())
    out_ds.SetGeoTransform(in_ds.GetGeoTransform())
    out_band = out_ds.GetRasterBand(1)
    if nodata is not None:
        out_band.SetNoDataValue(nodata)
    out_band.WriteArray(data)
    out_band.FlushCache()
    out_band.ComputeStatistics(False)
    return out_ds


**Note**: This dataset is from the National Agriculture Imagery Program (NAIP), part of the United States Department of Agriculture

In [ ]:
os.chdir(os.path.join(data_dir, 'Massachusetts'))
in_fn = 'm_4207162_ne_19_1_20140718_20140923_clip.tif'
out_fn = 'ndvi.tif'

In [ ]:
ds = gdal.Open(in_fn)

# The first band is red light, and the fourth is near-infrared, which is why you read these two bands in at the beginning
red = ds.GetRasterBand(1).ReadAsArray().astype(np.float)
nir = ds.GetRasterBand(4).ReadAsArray()

In [ ]:
# Mask out the red array in all locations where the sum of the two arrays is 0.
red = np.ma.masked_where(nir + red == 0, red)
ndvi = (nir - red) / (nir + red)

In [ ]:
#Fill the empty cells
ndvi = ndvi.filled(-99)

# Set NoData to the fill value when creating the new raster.
out_ds = pb.make_raster(ds, out_fn, ndvi, gdal.GDT_Float32, -99)

In [ ]:
overviews = compute_overview_levels(out_ds.GetRasterBand(1))
out_ds.BuildOverviews('average', overviews)
del ds, out_ds

### Focal Analyses
 * Focal analyses use the pixels that surround the target pixel in order to calculate a value
 * For a given cell in the output, the value is calculated based on the corresponding cell and its neighbors in the input dataset
 * Once the value for the target pixel is calculated, the window moves to the next pixel.
 * Focal analyses can also be used for anything else that requires input from surrounding pixels, such as computing slope and aspect for an elevation dataset.

<img src="images/moving_window_analysis.png" width =440 height = 150/>
             
                     moving window that calculates the average value of the nine surrounding pixels 

In [ ]:
import numpy as np
indata  = np.array([
    [3, 5, 6, 4, 4, 3],
    [4, 5, 8, 9, 6, 5],
    [2, 2, 5, 7, 6, 4],
    [5, 7, 9, 8, 9, 7],
    [4, 6, 5, 7, 7, 5],
    [3, 2, 5, 3, 4, 4]])

In [ ]:
outdata =  np.zeros((6, 6))

# Average of the nine surrounding pixels
outdata[2,2] = (indata[1,1] + indata[1,2] + indata[1,3] +
                indata[2,1] + indata[2,2] + indata[2,3] +
                indata[3,1] + indata[3,2] + indata[3,3]) / 9
print(outdata)

In [ ]:
# Shorter way to write the same thing

outdata[2,2] = np.mean(indata[1:4, 1:4])
print(outdata)

In [ ]:
# DO NOT try this on a real image because it's way too slow.
rows, cols = indata.shape
outdata = np.zeros(indata.shape, np.float32)
for i in range(1, rows-1):
    for j in range(1, cols-1):
        outdata[i,j] = np.mean(indata[i-1:i+2, j-1:j+2])
print(outdata)

**Note**: If the slices are all stacked into a three-dimensional array, then you can use the mean function, which would definitely be simpler. The `dstack` function will stack the slices on top of each other, which is what you need. But you still need to get all of the slices so you can pass them to `dstack`.

In [ ]:
# Check out some slices
slices = []
for i in range(3):
    for j in range(3):
        slices.append(indata[i:rows-2+i, j:cols-2+j])
print(slices)

# This is the upper left slice.
print(slices[0])

In [ ]:
# Stack the slices in the third dimension and compute the mean.
stacked = np.dstack(slices)
outdata = np.zeros(indata.shape, np.float32)
outdata[1:-1, 1:-1] = np.mean(stacked, 2)
print(outdata)

In [ ]:
# Function to get slices of any size from an array.
def make_slices(data, win_size):
    """Return a list of slices given a window size.

    data     - two-dimensional array to get slices from
    win_size - tuple of (rows, columns) for the moving window
    """
    # Calculate the slice size
    rows = data.shape[0] - win_size[0] + 1
    cols = data.shape[1] - win_size[1] + 1
    slices = []

    # Loop through the rows and columns in the provided window size and
    # create each slice.
    for i in range(win_size[0]):
        for j in range(win_size[1]):
            slices.append(data[i:rows+i, j:cols+j])
    return slices

In [ ]:
# Script to smooth an elevation dataset.
in_fn = os.path.join(data_dir, 'Nepal', 'everest.tif')
out_fn = os.path.join(data_dir, 'Nepal','everest_smoothed_edges.tif')

in_ds = gdal.Open(in_fn)
in_band = in_ds.GetRasterBand(1)
in_data = in_band.ReadAsArray()

# Stack the slices
slices = make_slices(in_data, (3, 3))
stacked_data = np.ma.dstack(slices)

rows, cols = in_band.YSize, in_band.XSize

# Initialize an output array to the NoData value (-99)
out_data = np.ones((rows, cols), np.int32) * -99

# Put the result into the middle of the output, leaving the
# outside rows and columns alone, so they still have -99.
out_data[1:-1, 1:-1] = np.mean(stacked_data, 2)

make_raster(in_ds, out_fn, out_data, gdal.GDT_Int32, -99)
del in_ds

Nothing is stopping you from applying much more complicated functions to the cells that make up the moving window. In fact, this is exactly what you’d want to do for many analyses.
 * One example is computing slope from an elevation model

<img src="images/slope_algorithm.png" width =440 height = 150/>

The next listing shows code for calculating the slope of the Mt. Everest DEM using these equations. 

**Note**: for this algorithm to work properly, the elevation units must be the same as the horizontal ones. For example, if your dataset uses a UTM projection, then the coordinates are expressed in meters, so the elevation values must also be meters.

In [ ]:
# Compute slope from DEM

in_fn = os.path.join(data_dir, 'Nepal', 'everest_utm.tif')
out_fn = os.path.join(data_dir,'Nepal', 'everest_slope.tif')

in_ds = gdal.Open(in_fn)
cell_width = in_ds.GetGeoTransform()[1]
cell_height = in_ds.GetGeoTransform()[5]
band = in_ds.GetRasterBand(1)
in_data = band.ReadAsArray().astype(np.float)

# Initialize output array with -99
slices = os.pymake_slices(in_data, (3, 3))

rise = ((slices[6] + (2 * slices[7]) + slices[8]) - (slices[0] + (2 * slices[1]) + slices[2])) / (8 * cell_height)
run = ((slices[2] + (2 * slices[5]) + slices[8]) - (slices[0] + (2 * slices[3]) + slices[6])) / (8 * cell_width)

# Output edges don’t get slope data
dist = np.sqrt(np.square(rise) + np.square(run))
out_data[1:-1, 1:-1] = np.arctan(dist) * 180 / np.pi

make_raster(in_ds, out_fn, out_data, gdal.GDT_Float32, -99)
del in_ds

#### Using SciPy for Focal Analysis

SciPy is a versatile Python module designed for scientific data analysis, and it uses NumPy arrays to store large amounts of data
 * It has submodules for interpolation, Fourier transforms, linear algebra, statistics, signal processing, and image processing, among others.
 * The multidimensional image processing submodule contains filtering functions that can be used to perform the same operations you did with NumPy.
**Note**: One advantage to using SciPy is that it will handle the edge problems for you by filling in extra cells around the edges so that the calculations can be performed on all cells

In [ ]:
# Smoothing filter using SciPy
import scipy.ndimage

in_fn = os.path.join(data_dir, 'Nepal', 'everest.tif')
out_fn = os.path.join(data_dir, 'Nepal', 'everest_smoothed.tif')
in_ds = gdal.Open(in_fn)
in_data = in_ds.GetRasterBand(1).ReadAsArray()

# Run the filter
out_data = scipy.ndimage.filters.uniform_filter(in_data, size=3, mode='nearest')
make_raster(in_ds, out_fn, out_data, gdal.GDT_Int32)
del in_ds

In [ ]:
# Calculate slope using SciPy
in_fn = os.path.join(data_dir, 'Nepal', 'everest_utm.tif')
out_fn = os.path.join(data_dir, 'Nepal', 'everest_slope_scipy2.tif')

def slope(data, cell_width, cell_height):
    """Calculates slope using a 3x3 window.
    data - 1D array containing the 9 pixel values, startingin the upper left and going left to right and down
    cell_width - pixel width in the same units as the data
    cell_height - pixel height in the same units as the data
    """
    rise = ((data[6] + (2 * data[7]) + data[8]) - (data[0] + (2 * data[1]) + data[2])) / (8 * cell_height)
    run = ((data[2] + (2 * data[5]) + data[8]) - (data[0] + (2 * data[3]) + data[6])) / (8 * cell_width)

    dist = np.sqrt(np.square(rise) + np.square(run))
    return np.arctan(dist) * 180 / np.pi

In [ ]:
in_ds = gdal.Open(in_fn)
in_band = in_ds.GetRasterBand(1)
in_data = in_band.ReadAsArray().astype(np.float32)

In [ ]:
cell_width = in_ds.GetGeoTransform()[1]
cell_height = in_ds.GetGeoTransform()[5]

# Run the filter
out_data = scipy.ndimage.filters.generic_filter( 
                                in_data, slope, size=3, mode='nearest', extra_arguments=(cell_width, cell_height)
                            )

In [ ]:
make_raster(in_ds, out_fn, out_data, gdal.GDT_Float32)
del in_ds